# Create proper dataset for learning

Out of many different CSV files record parametres of the platform during docking, only valueble information has to be extracted. 
Those informations are:
- Distance form the wall
- Rotation
- Distance setpoint
- Distance needed for docking
- Time of docking
- Base speed

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import random

In [4]:
file = 'ride_5'
path = '../scripts/logs/500_2/' + file + '.csv'

df_procedure = pd.read_csv(path, skiprows=[0,1,2, 3], names=['time[s]',
                                                          'front[mm]', 
                                                          'rear[mm]', 
                                                          'PID Align setpoint', 
                                                          'PID Distance setpoint', 
                                                          'error[mm]', 
                                                          'angle[rad]', 
                                                          'distance[mm]', 
                                                          'rw_speed[rad/s]', 
                                                          'lw_speeed[rad/s]', 
                                                          'pozyx_x_1', 
                                                          'pozyx_y_1',
                                                          'pozyx_rot_y_1', 
                                                          'pozyx_x_2',
                                                          'pozyx_y_2',
                                                          'pozyx_rot_2'
                                                        ])
df_single = pd.read_csv(path, skiprows=lambda x: x>2, index_col=0, header=None).T


In [8]:
print(df_procedure.head())

        time[s]  front[mm]  rear[mm]  PID Align setpoint  \
0  1.644503e+09      195.0     204.0           -0.156344   
1  1.644503e+09      195.0     204.0           -0.148911   
2  1.644503e+09      197.0     204.0           -0.148000   
3  1.644503e+09      202.0     204.0           -0.148000   
4  1.644503e+09      202.0     202.0           -0.148000   

   PID Distance setpoint  error[mm]  angle[rad]  distance[mm]  \
0                  500.0        9.0    0.046121    343.655573   
1                  500.0        9.0    0.046121    351.089227   
2                  500.0        7.0    0.035882    352.000000   
3                  500.0        2.0    0.010256    352.000000   
4                  500.0        0.0    0.000000    352.000000   

   rw_speed[rad/s]  lw_speeed[rad/s]  pozyx_x_1  pozyx_y_1  pozyx_rot_y_1  \
0              0.0               0.0      1.757       1.35      -0.169922   
1              0.0               0.0      1.757       1.35      -0.169922   
2              0.

In [10]:
base_speed = df_single.iloc[0,2]

# Procdure
front_sensor = df_procedure.iloc[0, 1]
rear_sensor = df_procedure.iloc[0, 2]
pid_align_setpoint = df_procedure.iloc[0, 3]
pid_distance_setpoint = df_procedure.iloc[0, 4]
error = df_procedure.iloc[0, 5]
angle = df_procedure.iloc[0, 6]
distance = df_procedure.iloc[0, 7]

# Single
docking_time = df_single.iloc[0,0]
docking_distance = df_single.iloc[0,1]

row = [base_speed,
       front_sensor,
       rear_sensor, 
       pid_align_setpoint, 
       pid_distance_setpoint,
       error, 
       angle, 
       distance,
       docking_time, 
       docking_distance
       ]

print(f'{base_speed=},{front_sensor=}, {rear_sensor=}, {pid_align_setpoint=}, {pid_distance_setpoint=}, {error=}, {angle=}, {distance=}, {docking_time=}, {docking_distance=}')

base_speed=2.5,front_sensor=195.0, rear_sensor=204.0, pid_align_setpoint=-0.1563444273730264, pid_distance_setpoint=500.0, error=9.0, angle=0.0461211160145112, distance=343.6555726269736, docking_time=18.49069333076477, docking_distance=2431.0


In [2]:
def clear_data(df):
        start_time = df.iloc[0,7]
        df = df[df['distance[mm]'] > 150.0]
        df['time[s]'] = df['time[s]'] - start_time
        df = df.apply(pd.to_numeric)

        return df

In [18]:
path = 'training_data_2.csv'

In [12]:
path = 'extended_training_data_2.csv'

In [13]:
def add_noise(value):
    value += np.random.normal(0, 1) * value/100
    return value

In [14]:
add_noise(1)

0.9939767975405991

In [20]:
f = open(path, 'w')
writer = csv.writer(f)


writer.writerow(['base_speed',
                'front[mm]', 
                'rear[mm]', 
                'PID Align setpoint', 
                'PID Distance setpoint', 
                'error[mm]', 
                'angle[rad]', 
                'distance[mm]', 
                'docking_time', 
                'docking_distance'
            ])


for n in range(25):

    try:
        file = 'ride_' + str(n)  + '.csv'
        path = '../scripts/logs/500_2/' + file


        df_procedure = pd.read_csv(path, skiprows=[0,1,2, 3], names=['time[s]',
                                                          'front[mm]', 
                                                          'rear[mm]', 
                                                          'PID Align setpoint', 
                                                          'PID Distance setpoint', 
                                                          'error[mm]', 
                                                          'angle[rad]', 
                                                          'distance[mm]', 
                                                          'rw_speed[rad/s]', 
                                                          'lw_speeed[rad/s]', 
                                                          'pozyx_x_1', 
                                                          'pozyx_y_1',
                                                          'pozyx_rot_y_1', 
                                                          'pozyx_x_2',
                                                          'pozyx_y_2',
                                                          'pozyx_rot_2'
                                                        ])
        df_single = pd.read_csv(path, skiprows=lambda x: x>2, index_col=0, header=None).T

        base_speed = df_single.iloc[0,2]

        # Procdure
        front_sensor = df_procedure.iloc[0, 1]
        rear_sensor = df_procedure.iloc[0, 2]
        pid_align_setpoint = df_procedure.iloc[0, 3]
        pid_distance_setpoint = df_procedure.iloc[0, 4]
        error = df_procedure.iloc[0, 5]
        angle = df_procedure.iloc[0, 6]
        distance = df_procedure.iloc[0, 7]

        # Single
        docking_time = df_single.iloc[0,0]
        docking_distance = df_single.iloc[0,1]

        row = [base_speed,
            front_sensor,
            rear_sensor, 
            pid_align_setpoint, 
            pid_distance_setpoint,
            error, 
            angle, 
            distance,
            docking_time, 
            docking_distance
            ]

        print(row)
        writer.writerow(row)
    except:
        print(n)


f.close()

[2.5, 103.0, 98.0, -0.2458527631418588, 500.0, -5.0, -0.0256354085216774, 254.1472368581412, 10.419591426849363, 1355.0]
[2.5, 679.0, 692.0, 0.3, 500.0, 13.0, 0.0665681637758238, 811.6792369444722, 9.662449836730955, 1320.0]
[2.5, 439.0, 439.0, 0.0855461467153579, 500.0, 0.0, 0.0, 585.546146715358, 6.75080156326294, 870.0]
3
[2.5, 522.0, 526.0, 0.1692721603430832, 500.0, 4.0, 0.0205099441394465, 669.2721603430832, 9.610530138015749, 1200.0]
[2.5, 195.0, 204.0, -0.1563444273730264, 500.0, 9.0, 0.0461211160145112, 343.6555726269736, 18.49069333076477, 2431.0]
[2.5, 659.0, 668.0, 0.3, 500.0, 9.0, 0.0461211160145112, 800.9993109335679, 10.292529344558716, 1377.0]
7
[2.5, 270.0, 259.0, -0.0721497243110861, 500.0, -11.0, -0.0563505357126421, 427.8502756889139, 5.187043905258179, 641.0]
[2.5, 520.0, 527.0, 0.1640088275786613, 500.0, 7.0, 0.0358820283531573, 664.0088275786613, 7.082317590713501, 908.0]
[2.5, 752.0, 763.0, 0.3, 500.0, 11.0, 0.0563505357126421, 885.4320020129604, 14.484589338302

In [235]:
path = 'extended_training_data.csv'
f = open(path, 'w')
writer = csv.writer(f)


writer.writerow(['base_speed',
                 'distance_from_wall', 
                 'rotation_angle', 
                 'distance_setpoint', 
                 'docking_time', 
                 'docking_distance'])


for n in range(49):

    try:
        file = 'ride_' + str(n)  + '.csv'
        path = '../scripts/logs/measurements/' + file
        df = pd.read_csv(path)

        docking_time = df.iloc[-3,1]
        docking_distance = df.iloc[-2,1]
        base_speed = df.iloc[-1,1]
        df = clear_data(df)
        distance_from_wall = df.iloc[0,6]
        rotation_angle = df.iloc[0, 5]
        distance_setpoint = df.iloc[0,3]

        for m in range(100):

            row = [base_speed,
            add_noise(distance_from_wall),
            add_noise(rotation_angle), 
            distance_setpoint, 
            add_noise(docking_time), 
            add_noise(docking_distance)]

            # print(row)
            writer.writerow(row)
    except:
        print(n)


f.close()

/tmp/ipykernel_37068/3733329275.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time[s]'] = df['time[s]'] - start_time


18
26
27
48
